In [6]:
import json
import zipfile
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import math
import csv
import operator
import Geohash
import random
from ForestBuilding import ForestBuilding
from build import TreeBuilding
from sklearn.cross_validation import train_test_split
import pickle
    
%matplotlib inline

In [7]:
air_trajs = pd.read_csv('1_500_data.csv',
                        #index_col=,
                        usecols = ['POLYLINE', 'Destination'],
                        converters={'POLYLINE': lambda x: json.loads(x), 'Destination': lambda x: json.loads(x)})

trajectories = list(air_trajs['POLYLINE'])
targets = list(air_trajs['Destination'])

X_train, X_test, y_train, y_test = train_test_split(trajectories, targets, train_size=0.10, random_state = 0)



In [8]:
len(X_train)

342

In [9]:
forest_clf = ForestBuilding()
forest = forest_clf.fit(X_train, y_train, 5, 1, 10, processes=2)

[<multiprocessing.pool.ApplyResult object at 0x10d276ad0>, <multiprocessing.pool.ApplyResult object at 0x10d276b50>, <multiprocessing.pool.ApplyResult object at 0x10d276a10>, <multiprocessing.pool.ApplyResult object at 0x10d276850>, <multiprocessing.pool.ApplyResult object at 0x10d2768d0>]


PicklingError: Can't pickle <type 'instancemethod'>: attribute lookup __builtin__.instancemethod failed

In [5]:
forest.score(X_test, y_test)

105.93233362996237

In [5]:
tree_clf = TreeBuilding()
tree = tree_clf.fit(X_train, y_train, 1, 10)

In [6]:
tree.score(X_test, y_test)

437.22908562009326

In [5]:
test_trajs = pd.read_csv('test.csv',
                        #index_col=,
                        usecols = ['TRIP_ID', 'POLYLINE'],
                        converters={'POLYLINE': lambda x: json.loads(x)})

In [6]:
prediction = forest.predict(test_trajs['POLYLINE'])

In [7]:
arr = np.array(prediction)

test_trajs['LATITUDE'] = np.array(arr[:,1])
test_trajs['LONGITUDE'] = np.array(arr[:,0])

In [8]:
df_pred = test_trajs.drop('POLYLINE', 1)
df_pred.to_csv('test_submission_500_forest.csv', index = False)

In [7]:
pickle.dump(forest, open( "forest_500_data_50_trees.p", "wb" ))

RuntimeError: maximum recursion depth exceeded